In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import yaml

# Prepare directories

In [40]:
BASE_FILE_PATH = Path("../../datasets/drosophila_enhancers_stark/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)
    
BASE_FILE_PATH.mkdir()
(BASE_FILE_PATH / 'train').mkdir()
(BASE_FILE_PATH / 'test').mkdir()

# Positive samples

## Download enhancers

In [41]:
!wget https://enhancers.starklab.org/download/tiles/all -O tiles.csv

--2022-06-29 15:03:23--  https://enhancers.starklab.org/download/tiles/all
Resolving enhancers.starklab.org (enhancers.starklab.org)... 193.171.188.109
Connecting to enhancers.starklab.org (enhancers.starklab.org)|193.171.188.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957907 (935K) [text/csv]
Saving to: ‘tiles.csv’

tiles.csv           100%[===================>] 935,46K   854KB/s    in 1,1s    

2022-06-29 15:03:24 (854 KB/s) - ‘tiles.csv’ saved [957907/957907]



## Filter only verified enhancers

In [42]:
df = pd.read_csv('tiles.csv')
df

,VTID,Chrosome,Start,End,Length,Verification Status,Positive,stg4_6,stg7_8,stg9_10,stg11_12,stg13_14,stg15_16
0,VT0002,chr2L,9410,10052,642,correct,0,NaN,NaN,NaN,NaN,NaN,NaN
1,VT0003,chr2L,11245,11850,605,correct,1,NaN,NaN,NaN,NaN,NaN,hindgut;2|posterior_hindgut;3
2,VT0004,chr2L,12730,13724,994,correct,0,NaN,NaN,NaN,NaN,NaN,NaN
3,VT0005,chr2L,15507,17836,2329,correct,0,NaN,NaN,NaN,NaN,dorsal_epidermis_subset;1,dorsal_epidermis_subset;1
4,VT0006,chr2L,16836,18924,2088,correct,1,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;3,brain_broad;3|ubiquitous;3|ventral_nerve_cord_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,VT9951,chr2L,19419622,19421726,2104,correct,1,NaN,NaN,NaN,NaN,lateral_epidermis_subset;3|ventral_epidermis_s...,head_epidermis_ventral_subset;3|lateral_epider...
7789,VT9978,chr2L,19469452,19471821,2369,correct,1,NaN,NaN,NaN,NaN,ventral_nerve_cord_subset;2|brain_subset;2,ventral_nerve_cord_subset;2|brain_subset;2
7790,VT9983,chr2L,19477513,19479654,2141,correct,1,NaN,NaN,NaN,NaN,ventral_nerve_cord_subset;1|brain_subset;1,ventral_nerve_cord_subset;1|midgut_chamber;2|b...
7791,VT9998,chr2L,19510549,19511569,1020,correct,0,NaN,NaN,NaN,NaN,NaN,NaN


### Verification status

From https://enhancers.starklab.org/methods: "The identity of 7478 lines (96%) was confirmed using PCR of genomic DNA isolated from transgenic flies followed by Sanger sequencing (the remaining 4% were either not tested or failed)."

In [43]:
df['Verification Status'].value_counts()

correct         7478
not_verified     315
Name: Verification Status, dtype: int64

### Positive

From https://www.nature.com/articles/nature13395: We also manually annotated the strength of the signal for each annotation term semiquantitatively from 1 (very weak) to 5 (very strong). A line was considered as positive if it contained at least one annotation term with strength >1 (excluding enhancers with only very weak (1) activities). This resulted in 3,557 positively annotated VT strains (4,480 (58.1%) if also considering very weak signals).


In [44]:
df['Positive'].value_counts()

0    4189
1    3604
Name: Positive, dtype: int64

In [45]:
df = df[(df['Verification Status'] == 'correct') & (df['Positive'] == 1)]
df = df.reset_index()
df

,index,VTID,Chrosome,Start,End,Length,Verification Status,Positive,stg4_6,stg7_8,stg9_10,stg11_12,stg13_14,stg15_16
0,1,VT0003,chr2L,11245,11850,605,correct,1,NaN,NaN,NaN,NaN,NaN,hindgut;2|posterior_hindgut;3
1,4,VT0006,chr2L,16836,18924,2088,correct,1,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;3,brain_broad;3|ubiquitous;3|ventral_nerve_cord_...
2,5,VT0007,chr2L,18031,20238,2207,correct,1,ubiquitous;3,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;4,midgut_chamber;4|ubiquitous;4
3,10,VT0013,chr2L,26520,27246,726,correct,1,procephalic_ectoderm_AISN;4,procephalic_ectoderm_anlage;3|head_mesoderm_an...,NaN,NaN,NaN,NaN
4,19,VT0025,chr2L,54047,56104,2057,correct,1,NaN,NaN,NaN,NaN,NaN,antenno-maxillary-labial_complex;4|ventral_epi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452,7784,VT9946,chr2L,19410749,19412957,2208,correct,1,NaN,NaN,NaN,amnioserosa;2,amnioserosa;3,amnioserosa;3
3453,7786,VT9949,chr2L,19415569,19417675,2106,correct,1,NaN,NaN,NaN,hindgut_proper_primordium;1|posterior_midgut_p...,hindgut;2|midgut_subset;2,head_subset;3|midgut_broad;3|hindgut;3|salivar...
3454,7788,VT9951,chr2L,19419622,19421726,2104,correct,1,NaN,NaN,NaN,NaN,lateral_epidermis_subset;3|ventral_epidermis_s...,head_epidermis_ventral_subset;3|lateral_epider...
3455,7789,VT9978,chr2L,19469452,19471821,2369,correct,1,NaN,NaN,NaN,NaN,ventral_nerve_cord_subset;2|brain_subset;2,ventral_nerve_cord_subset;2|brain_subset;2


In [46]:
df = df[['Chrosome', 'Start', 'End']]
df.columns = ['region', 'start', 'end']
df['strand'] = '+'
df

/tmp/ipykernel_381321/3624373002.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['strand'] = '+'


,region,start,end,strand
0,chr2L,11245,11850,+
1,chr2L,16836,18924,+
2,chr2L,18031,20238,+
3,chr2L,26520,27246,+
4,chr2L,54047,56104,+
...,...,...,...,...
3452,chr2L,19410749,19412957,+
3453,chr2L,19415569,19417675,+
3454,chr2L,19419622,19421726,+
3455,chr2L,19469452,19471821,+


## Lift to dm6 genome assembly

In [47]:
!pip install pyliftover

You should consider upgrading via the '/home/katarina/git/genomic_benchmarks/venv/bin/python -m pip install --upgrade pip' command.


In [48]:
from pyliftover import LiftOver
lo = LiftOver('dm3', 'dm6')

In [49]:
def convert_coordinate(region, start, end, strand):
    results = lo.convert_coordinate(region, start, strand)
    if results is None:
        return []
    else:
        reg, sta, str, _ = results[0]
        en = sta + (end - start)
        return [reg, sta, en, str]

In [50]:
df['region'].value_counts()

chr3R       1004
chr2L        658
chr2R        636
chr3L        618
chrX         526
chr4          14
chr3LHet       1
Name: region, dtype: int64

In [51]:
df = pd.DataFrame(df.apply(lambda x:
    convert_coordinate(x['region'], x['start'], x['end'], x['strand']),
    axis=1
).tolist(), columns=['region', 'start', 'end', 'strand'])
df

,region,start,end,strand
0,chr2L,11245,11850,+
1,chr2L,16836,18924,+
2,chr2L,18031,20238,+
3,chr2L,26520,27246,+
4,chr2L,54047,56104,+
...,...,...,...,...
3452,chr2L,19410749,19412957,+
3453,chr2L,19415569,19417675,+
3454,chr2L,19419622,19421726,+
3455,chr2L,19469452,19471821,+


In [52]:
df['region'].value_counts()

chr3R    1004
chr2L     658
chr2R     636
chr3L     619
chrX      526
chr4       14
Name: region, dtype: int64

## Filtering chromosomes



In [53]:
CHROMOSOMES = ['chr2L', 'chr2R', 'chr3L', 'chr3R', 'chr4', 'chrX', 'chrY', 'chrM']

In [54]:
df['region'].value_counts()

chr3R    1004
chr2L     658
chr2R     636
chr3L     619
chrX      526
chr4       14
Name: region, dtype: int64

In [55]:
df = df[df['region'].isin(CHROMOSOMES)]
df = df.reset_index(drop=True)

In [56]:
df['region'].value_counts()

chr3R    1004
chr2L     658
chr2R     636
chr3L     619
chrX      526
chr4       14
Name: region, dtype: int64

## Train-test split

In [57]:
df.index.name = "id"
train_pos_seqs, test_pos_seqs = train_test_split(df, shuffle=True, random_state=42)
train_pos_seqs.shape, test_pos_seqs.shape

((2592, 4), (865, 4))

# Generating negative samples

In [58]:
import os
from twobitreader.download import save_genome
from twobitreader import TwoBitFile

def get_2bit_genome_file(genome_name, local_dir):
    save_genome(genome_name, destdir=local_dir, mode='http')
    twobit_path = os.path.join(local_dir, genome_name + '.2bit')
    return TwoBitFile(twobit_path)

In [59]:
def get_chr_names_and_lengths():

    chr_lengths = {}
    genome = get_2bit_genome_file('dm6', './')
    for chromosome in genome.keys():
        if chromosome in CHROMOSOMES:
            chr_lengths[chromosome] = len(genome[chromosome])

    # check that all lengths are different from 0
    assert all(x != 0 for x in chr_lengths.values())

    return chr_lengths

In [60]:
def get_random_chr(chr_names_and_lengths):
    chr_lengths = pd.Series(chr_names_and_lengths.values())
    chr_probs = chr_lengths / chr_lengths.sum()
    chr_names = list(chr_names_and_lengths.keys())
    return chr_names[np.argwhere(np.random.multinomial(1, chr_probs))[0][0]]


def is_intersecting(c, pos, df_forbidden):
    intersecting = (df_forbidden['region'] == c) & (df_forbidden['start'].astype(int) <= pos) & (
                df_forbidden['end'].astype(int) >= pos)
    return intersecting.any()


def get_random_pos(df_forbidden: pd.DataFrame, chr_names_and_lengths, offset_from_end):
    c = get_random_chr(chr_names_and_lengths)
    c_len = chr_names_and_lengths[c]
    pos = np.random.randint(c_len - offset_from_end) + 1

    while is_intersecting(c, pos, df_forbidden):
        pos = np.random.randint(c_len) + 1

    return c, pos

In [61]:
chr_names_and_lengths = get_chr_names_and_lengths()
chr_names_and_lengths

{'chr2L': 23513712,
 'chr2R': 25286936,
 'chr3L': 28110227,
 'chr3R': 32079331,
 'chr4': 1348131,
 'chrM': 19524,
 'chrX': 23542271,
 'chrY': 3667352}

In [62]:
get_random_pos(df, chr_names_and_lengths, 0)

('chr2R', 17364976)

In [63]:
num_seqs = len(df)
negative_samples = [None] * num_seqs
genome = get_2bit_genome_file('dm6', './')
#seqs = [None] * num_seqs
for i in range(num_seqs):
    while True:
        seq_length = int(df['end'][i]) - int(df['start'][i])
        chrom, start = get_random_pos(df, chr_names_and_lengths, seq_length)
        end = start + seq_length
        seq = genome[chrom][start:end]
        if 'N' not in seq.upper():
            negative_samples[i] = [chrom, start, end, '+']
            break
df = pd.DataFrame(negative_samples, columns=['region', 'start', 'end', 'strand'])
df

,region,start,end,strand
0,chr2R,2580688,2581293,+
1,chr3R,20787763,20789851,+
2,chr2R,13029315,13031522,+
3,chrX,18148359,18149085,+
4,chr2R,9553093,9555150,+
...,...,...,...,...
3452,chr2R,11505900,11508108,+
3453,chr3L,505843,507949,+
3454,chr2R,3437488,3439592,+
3455,chr3R,28857544,28859913,+


## Train-test split

In [64]:
df.index.name = "id"
train_neg_seqs, test_neg_seqs = train_test_split(df, shuffle=True, random_state=42)
train_neg_seqs.shape, test_neg_seqs.shape

((2592, 4), (865, 4))

# YAML file

In [65]:
with open(BASE_FILE_PATH / 'metadata.yaml', 'w') as fw:
    desc = {
        'version': 0,
        'classes': {
            'positive': {
                'type': 'fa.gz',
                'url': 'http://ftp.ensembl.org/pub/release-100/fasta/drosophila_melanogaster/dna/Drosophila_melanogaster.BDGP6.28.dna.toplevel.fa.gz',
                'extra_processing': 'ENSEMBL_DROSOPHILA_GENOME' 
            },    
            'negative': {
                'type': 'fa.gz',
                'url': 'http://ftp.ensembl.org/pub/release-100/fasta/drosophila_melanogaster/dna/Drosophila_melanogaster.BDGP6.28.dna.toplevel.fa.gz'
            }
        }
    }
    
    yaml.dump(desc, fw)

desc

{'version': 0,
 'classes': {'positive': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-100/fasta/drosophila_melanogaster/dna/Drosophila_melanogaster.BDGP6.28.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_DROSOPHILA_GENOME'},
  'negative': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-100/fasta/drosophila_melanogaster/dna/Drosophila_melanogaster.BDGP6.28.dna.toplevel.fa.gz'}}}

# CSV files

In [66]:
train_pos_seqs.to_csv(BASE_FILE_PATH / 'train' / 'positive.csv.gz', index=True, compression='gzip')
train_neg_seqs.to_csv(BASE_FILE_PATH / 'train' / 'negative.csv.gz', index=True, compression='gzip')
test_pos_seqs.to_csv(BASE_FILE_PATH / 'test' / 'positive.csv.gz', index=True, compression='gzip')
test_neg_seqs.to_csv(BASE_FILE_PATH / 'test' / 'negative.csv.gz', index=True, compression='gzip')

# Cleanup

In [70]:
!rm ./dm6.2bit
!rm tiles.csv

# Test that it can be downloaded

In [74]:
from genomic_benchmarks.loc2seq import download_dataset

download_dataset("drosophila_enhancers_stark", use_cloud_cache=False, force_download=True, local_repo=True)

/home/katarina/git/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


/home/katarina/.genomic_benchmarks/fasta/Drosophila_melanogaster.BDGP6.28.dna.toplevel.fa.gz: 43.3MB [00:10, 4.12MB/s]                            


/home/katarina/.genomic_benchmarks/fasta/Drosophila_melanogaster.BDGP6.28.dna.toplevel.fa.gz: 43.3MB [00:10, 4.14MB/s]                            
1870it [00:01, 1300.62it/s]
  0%|          | 7/1870 [00:01<06:08,  5.05it/s]


PosixPath('/home/katarina/.genomic_benchmarks/drosophila_enhancers_stark')

In [69]:
from genomic_benchmarks.data_check import info

info("drosophila_enhancers_stark", 0, local_repo=True)

Dataset `drosophila_enhancers_stark` has 2 classes: negative, positive.

The length of genomic intervals ranges from 236 to 3237, with average 2118.1238067688746 and median 2142.0.

Totally 6914 sequences have been found, 5184 for training and 1730 for testing.


,train,test
negative,2592,865
positive,2592,865
